In [1]:
import pandas as pd
import numpy as np
import time
import requests

### Grabs the players data using the fetchplayerstats class
Enter the year you want in season and if you want Regualr Season or Playoffs in season_type

In [2]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2024-25', season_type='Playoffs', sleep_time=1, max_workers=5)
data

[1] Fetching basic player stats...
[2] Fetching advanced player stats...
Total games: 82, Cached: 82, To fetch: 0
[3] Merging player data...
[4] Fetching and processing team data...
[1/30] Fetching data for Atlanta Hawks
[2/30] Fetching data for Boston Celtics
[3/30] Fetching data for Cleveland Cavaliers


c:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\NBAData\fetchPlayersStats.py:24: SyntaxWarning: invalid escape sequence '\.'
  df['HOME_GAME'] = df['MATCHUP'].str.contains('vs\.').astype(int)


KeyboardInterrupt: 

In [5]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')

In [3]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')
data

,Unnamed: 0,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,0,Nickeil Alexander-Walker,1629638,MIN @ LAL,MIN,1610612750,LAL,0,42400161,2025-04-19,W,20,7,3,1,2,9,0.222,1,6,0.167,2,2,1.000,0,1,0,1,0,2,9,15.7,0.709,0.277778,NaN,142.1,121.6,20.5,0.000,0.050,0.026,0.176,0.0,0.222,0.354,88.94,89.04,0.029,74.20,"APR 19, 2025",240,44,86,0.512,21,42,0.500,8,10,0.800,11,33,44,29,8,4,10,21,117,130.872483,5.0,4.0,38.0,0.397590,1.610613e+09,130.872483,89.40,89.72,90.04
1,1,Michael Porter Jr.,1629008,DEN vs. LAC,DEN,1610612743,LAC,1,42400171,2025-04-19,W,26,3,0,4,1,4,0.250,1,2,0.500,0,0,NaN,1,3,0,0,1,1,-8,6.8,0.750,0.375000,F,92.3,112.0,-19.7,0.040,0.115,0.078,0.000,0.0,0.088,0.375,94.43,92.51,0.018,77.09,"APR 19, 2025",265,41,94,0.436,9,27,0.333,21,27,0.778,12,33,45,24,11,1,11,18,112,106.788711,7.0,6.0,44.0,0.500000,1.610613e+09,106.788711,104.88,103.46,102.04
2,2,Jalen Pickett,1629618,DEN vs. LAC,DEN,1610612743,LAC,1,42400171,2025-04-19,W,5,2,0,2,1,1,1.000,0,0,NaN,0,0,NaN,0,2,0,0,0,1,-8,4.4,2.000,1.000000,NaN,54.5,127.3,-72.7,0.000,0.400,0.154,0.000,0.0,0.067,1.000,112.31,105.95,0.231,88.29,"APR 19, 2025",265,41,94,0.436,9,27,0.333,21,27,0.778,12,33,45,24,11,1,11,18,112,106.788711,7.0,6.0,44.0,0.500000,1.610613e+09,106.788711,104.88,103.46,102.04
3,3,Peyton Watson,1631212,DEN vs. LAC,DEN,1610612743,LAC,1,42400171,2025-04-19,W,14,6,0,2,2,4,0.500,0,1,0.000,2,2,1.000,0,2,1,0,1,2,-8,10.4,1.230,0.500000,NaN,63.3,81.8,-18.5,0.000,0.125,0.054,0.000,0.0,0.167,0.615,107.57,106.35,0.157,88.63,"APR 19, 2025",265,41,94,0.436,9,27,0.333,21,27,0.778,12,33,45,24,11,1,11,18,112,106.788711,7.0,6.0,44.0,0.500000,1.610613e+09,106.788711,104.88,103.46,102.04
4,4,Russell Westbrook,201566,DEN vs. LAC,DEN,1610612743,LAC,1,42400171,2025-04-19,W,34,15,3,8,5,17,0.294,2,6,0.333,3,4,0.750,4,4,2,0,1,3,7,34.1,0.800,0.352941,NaN,110.3,101.5,8.8,0.098,0.148,0.118,0.136,3.0,0.235,0.400,95.27,94.13,0.080,78.44,"APR 19, 2025",265,41,94,0.436,9,27,0.333,21,27,0.778,12,33,45,24,11,1,11,18,112,106.788711,7.0,6.0,44.0,0.500000,1.610613e+09,106.788711,104.88,103.46,102.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,1748,Kenrich Williams,1629026,OKC vs. IND,OKC,1610612760,IND,1,42400405,2025-06-16,W,11,0,1,1,0,1,0.000,0,1,0.000,0,0,NaN,0,1,0,0,0,0,4,2.7,0.000,0.000000,NaN,121.7,120.0,1.7,0.000,0.091,0.050,0.125,0.0,0.037,0.000,98.03,96.15,0.022,80.12,"JUN 16, 2025",240,40,94,0.426,14,32,0.438,26,32,0.813,19,26,45,24,15,12,11,24,120,119.904077,9.0,4.0,50.0,0.451220,1.610613e+09,119.904077,100.08,99.64,99.20
1749,1749,Tony Bradley,1628396,IND @ OKC,IND,1610612754,OKC,0,42400405,2025-06-16,L,12,4,0,4,1,2,0.500,0,0,NaN,2,2,1.000,2,2,0,0,1,3,2,7.8,1.389,0.500000,NaN,103.7,100.0,3.7,0.105,0.100,0.103,0.000,0.0,0.108,0.694,100.72,103.70,0.048,86.41,"JUN 16, 2025",240,37,82,0.451,11,30,0.367,24,30,0.800,18,32,50,23,9,4,22,25,109,109.879032,15.0,12.0,45.0,0.425532,1.610613e+09,109.879032,99.20,99.64,100.08
1750,1750,Isaiah Hartenstein,1628392,OKC vs. IND,OKC,1610612760,IND,1,42400405,2025-06-16,W,21,4,4,8,1,2,0.500,0,0,NaN,2,4,0.500,6,2,1,1,1,3,-1,24.6,1.064,0.500000,C,111.6,108.9,2.7,0.214,0.080,0

In [4]:
from NBAData.features import *
from NBAData.playerPositions import *

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = calculate_rolling_averages(data, [2, 4, 6])
data = home_away_averages(data)
data = calculate_days_of_rest(data)
# data = assign_playoff_series_info(data)
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data

Loading position cache...
Loaded 219 players from cache
Found 219 unique players, 0 need to be fetched
Applying positions to dataset...
Creating position flags...
Position assignment completed!


,Unnamed: 0,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,PTS_ROLL_AVG_2,PTS_STD_AVG_2,MIN_ROLL_AVG_2,FG_PCT_ROLL_AVG_2,FGM_ROLL_AVG_2,FGA_ROLL_AVG_2,FG3M_ROLL_AVG_2,FG3A_ROLL_AVG_2,FG3_PCT_ROLL_AVG_2,FTM_ROLL_AVG_2,FT_PCT_ROLL_AVG_2,REB_ROLL_AVG_2,AST_ROLL_AVG_2,USG_PCT_ROLL_AVG_2,PTS_ROLL_AVG_4,PTS_STD_AVG_4,MIN_ROLL_AVG_4,FG_PCT_ROLL_AVG_4,FGM_ROLL_AVG_4,FGA_ROLL_AVG_4,FG3M_ROLL_AVG_4,FG3A_ROLL_AVG_4,FG3_PCT_ROLL_AVG_4,FTM_ROLL_AVG_4,FT_PCT_ROLL_AVG_4,REB_ROLL_AVG_4,AST_ROLL_AVG_4,USG_PCT_ROLL_AVG_4,PTS_ROLL_AVG_6,PTS_STD_AVG_6,MIN_ROLL_AVG_6,FG_PCT_ROLL_AVG_6,FGM_ROLL_AVG_6,FGA_ROLL_AVG_6,FG3M_ROLL_AVG_6,FG3A_ROLL_AVG_6,FG3_PCT_ROLL_AVG_6,FTM_ROLL_AVG_6,FT_PCT_ROLL_AVG_6,REB_ROLL_AVG_6,AST_ROLL_AVG_6,USG_PCT_ROLL_AVG_6,PLAYER_HOME_AVG_PTS,PLAYER_AWAY_AVG_PTS,DAYS_OF_REST,GUARD,FORWARD,CENTER
34,34,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400161,2025-04-19,L,36,19,3,5,8,18,0.444,1,5,0.200,2,2,1.000,1,4,2,3,4,0,-22,40.5,1.006,0.472222,F,108.7,142.6,-34.0,0.029,0.138,0.078,0.158,0.75,0.278,0.503,90.65,90.70,0.099,75.59,"APR 19, 2025",240,33,83,0.398,15,41,0.366,14,16,0.875,13,25,38,15,5,4,13,10,95,105.508663,8.0,4.0,44.0,0.511628,1.610613e+09,105.508663,90.04,89.72,89.40,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.00,NaN,NaN,0,1,0
243,243,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400162,2025-04-22,W,40,21,7,11,8,19,0.421,1,5,0.200,4,4,1.000,3,8,1,1,2,4,9,48.7,1.012,0.447368,F,111.4,98.6,12.9,0.094,0.195,0.151,0.368,3.50,0.284,0.506,88.21,84.81,0.224,70.68,"APR 22, 2025",240,34,75,0.453,6,29,0.207,20,20,1.000,9,32,41,23,4,6,12,24,94,108.294931,6.0,2.0,34.0,0.379747,1.610613e+09,108.294931,86.80,89.40,92.00,1,19.0,NaN,36.0,0.44,8.0,18.0,1.0,5.0,0.20,2.0,1.00,5.0,3.0,0.28,19.00,NaN,36.00,0.44,8.00,18.00,1.00,5.00,0.20,2.00,1.00,5.00,3.00,0.28,19.00,NaN,36.00,0.44,8.00,18.00,1.00,5.00,0.20,2.00,1.00,5.00,3.00,0.28,20.00,NaN,3.0,0,1,0
463,463,LeBron James,2544,LAL @ MIN,LAL,1610612747,MIN,0,42400163,2025-04-25,L,41,38,4,10,13,21,0.619,5,9,0.556,7,10,0.700,3,7,2,2,1,0,-10,67.0,1.496,0.738095,F,109.9,117.9,-8.0,0.077,0.179,0.128,0.211,4.00,0.286,0.748,98.35,96.23,0.266,80.19,"APR 25, 2025",240,37,78,0.474,17,40,0.425,13,20,0.650,7,31,38,21,5,7,16,23,104,108.559499,10.0,5.0,38.0,0.483516,1.610613e+09,108.559499,95.80,97.52,99.24,1,20.0,1.41,38.0,0.43,8.0,18.5,1.0,5.0,0.20,3.0,1.00,8.0,5.0,0.28,20.00,1.41,38.00,0.43,8.00,18.50,1.00,5.00,0.20,3.00,1.00,8.00,5.00,0.28,20.00,1.41,38.00,0.43,8.00,18.50,1.00,5.00,0.20,3.00,1.00,8.00,5.00,0.28,NaN,38.00,3.0,0,1,0
600,600,LeBron James,2544,LAL @ MIN,LAL,1610612747,MIN,0,42400164,2025-04-27,L,46,27,8,12,5,9,0.556,2,4,0.500,15,18,0.833,3,9,3,3,3,2,-4,68.4,1.596,0.666667,F,119.1,125.0,-5.9,0.070,0.158,0.120,0.276,2.67,0.186,0.798,92.80,91.88,0.207,76.57,"APR 27, 2025",240,36,80,0.450,19,47,0.404,22,25,0.880,11,30,41,23,6,7,10,23,113,125.555556,8.0,2.0,49.0,0.422222,1.610613e+09,125.555556,90.00,92.76,95.52,1,29.5,12.02,40.5,0.52,10.5,20.0,3.0,7.0,0.38,5.5,0.85,10.5,5.5,0.28,26.00,10.44,39.00,0.49,9.67,19.33,2.33,6.33,0.32,4.33,0.90,8.67,4.67,0.28,26.00,10.44,39.00,0.49,9.67,19.33,2.33,6.33,0.32,4.33,0.90,8.67,4.67,0.28,NaN,32.50,2.0,0,1,0
777,777,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400165,2025-04-30,L,40,2